In [ ]:
"""  
  wget  https://chromedriver.storage.googleapis.com/83.0.4103.39/chromedriver_linux64.zip -O /tmp/chromedriver_linux64.zip
  mkdir /tmp/chromedriver
  unzip /tmp/chromedriver_linux64.zip -d /tmp/chromedriver/
  sudo add-apt-repository ppa:canonical-chromium-builds/stage
  /usr/bin/yes | sudo apt update
  /usr/bin/yes | sudo apt install chromium-browser
"""

In [ ]:
#Se utiliza selenium para dar click en el boton debido a que es una pagina dinamica
from selenium import webdriver
import pandas as pd
from lxml import html
import time

#Se levanda un webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_driver = "/tmp/chromedriver/chromedriver"
driver = webdriver.Chrome(executable_path=chrome_driver, chrome_options=chrome_options)

#Se ejecutan los pasos correspondientes para llegar a la tabla
driver.get('http://siga.inta.gov.ar/#/data')
#Se espera 5 segundos para que cargue la pagina
time.sleep(5)
driver.execute_script('document.getElementsByClassName("tabFont")[1].click();')
#Boton para descargar csv
#driver.execute_script('document.getElementById("btnDatosHorarios").click();')

#Luego de visualizar la tabla, se extrae el contenido html
page = html.fromstring(driver.page_source)
trs = page.xpath('/html/body/div[2]/div/div[2]/div/div[2]/div[2]/div[1]/div[2]/div/div/div[1]/div/table//tr')
print(len(trs))
#Se procesa el html para convertir la tabla a un df
df = None
if trs:
    items = []
    for tr in trs[1:]:
        #print(tr.xpath('td[1]//text()'), len(trs))
        item = {
                        "Nombre": tr.xpath('td[1]//text()')[0] if tr.xpath('td[1]//text()') else None,
                        "Tipo": tr.xpath('td[2]//text()')[0] if tr.xpath('td[2]//text()') else None,
                        "Localidad": tr.xpath('td[3]//text()')[0] if tr.xpath('td[3]//text()') else None,
                        "Provincia": tr.xpath('td[4]//text()')[0] if tr.xpath('td[4]//text()') else None,
                        "Id Interno": tr.xpath('td[5]//text()')[0] if tr.xpath('td[5]//text()') else None,
                        "Desde": tr.xpath('td[6]//text()')[0] if tr.xpath('td[6]//text()') else None,
                        "Hasta": tr.xpath('td[7]//text()')[0] if tr.xpath('td[7]//text()') else None,
                        "Días": tr.xpath('td[8]//text()')[0] if tr.xpath('td[8]//text()') else None,
                    }
        items.append(item)


        df = pd.DataFrame(items, columns=['Nombre','Tipo','Provincia','Id Interno',
                    'Desde','Hasta','Días'])

#Se almacena en el lake
df.to_csv('/dbfs/mnt/test/raw-data/automations/weather_station/Estaciones.csv')